In [1]:
import sys
import time
import torch
import torch.optim as optim
import torch.nn.functional as FTR
from torch.autograd import Variable
import pickle
import scipy as sp
from scipy import spatial
import numpy as np
from sklearn.cluster import KMeans
#from dge_fun import *
#from kmeans import lloyd



#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
torch.cuda.is_available()

# FORCE CPU for timing purpose
#device = torch.device("cpu")

#load cleaned data
path_dir = ""
save_path = ""
lossdata_path = ""



cuda


In [2]:
#--- FUNCTION DEFINITONS --> PUT INTO MODULE
#--- ? HOW TO PASS ON "device" TO MODULE ?

def class_sim_torch(X):
    Tpxy=torch.sum(torch.abs(X - X[:, None]),2)
    return Tpxy

def PDIST(X):
    #print(X.type())
    N=X.size()[0]
    dim=X.size()[1]
    numel=N*N*dim
    numel_max = 400000000
    #numel_max = 100000000
    nl=np.ceil(numel/numel_max).astype(int) # divide dim by this
    if nl ==1:
        Tpxy=torch.sum(torch.abs(X - X[:, None]),2)
    else:
        Tpxy=torch.zeros(N,N).to(device)
        Tpxy=Tpxy.type(X.type())
        mdim=np.floor(dim/nl).astype(int) # dim slices
        for i in range(nl+1):
            i1=i*mdim
            i2=np.minimum((i+1)*mdim,dim)
            if i1<i2:
                Xt=X[:,i1:i2]
                Tpxy.add_(torch.sum(torch.abs(Xt - Xt[:, None]),2))    
    return Tpxy

def NLSS( X, param,r_loc):
#    XSS_ij = exp( -1/param * sum_k |X(i,k) - X(j,k)| / N_features )
# size(X)= N_frames x N_features 
# size(XSS)= N_frames x N_frames 
#
# if r_loc is given, it specifies the "search radius" for NLmeans
# in this case:
#    XSS_ij is exponentially tapered
#    XSS_ij = 0 for |i-j|>r_loc
#    XSS is periodically extended to size N_frames x N_frames + 2*r_loc
    if len(sys.argv)<3:
        r_loc=[]
        
    N = len(X)
    dim = X.shape[1]
    #--- compute distance & similarity
    XM2 = sp.spatial.distance.squareform(sp.spatial.distance.pdist(X,metric = 'cityblock'))/dim
    XSS0 = np.exp(-1/param*XM2)

    # if not NLM over whole sequence, set similarity beyond search radius to
    # zero and extend boundaries
    if r_loc:   
        t=np.arange(1,N)
        for i in range(1,N):
            t = np.concatenate((t, np.arange(1,N-i)), axis=None)
        t = np.transpose(t)
        T=spatial.distance.squareform(t) #%T=T(:,1:min(N1,N2));
        MASK=sp.exp(math.log(0.25)*T/r_loc)
        XSS=np.multiply(XSS0,MASK)
        XSS = np.transpose(XSS)
        XSS= np.concatenate((np.flipud(XSS[0:r_loc,:]),XSS,np.flipud(XSS[XSS.shape[0]-r_loc:XSS.shape[0],:])), axis=0)
        XSS = np.transpose(XSS)
    return XSS

def construct_SS_matrix(X,nnh):
# function [XX] = construct_SS_matrix(X,nnh)
# 
# stack nnh left-and right neighbor feature vectors of X using periodic
# boundary conditions.  
# size(X)  = N_frames x N_features 
# size(XX) = N_frames x N_features* 2*nnh 

    # periodic border effects
    XE=np.concatenate((  np.flipud(X[1:nnh+1,:]),X,(np.flipud(X[X.shape[0]-nnh-2:X.shape[0]-1,:]))), axis = 0)
    
    #--- construct neighborhood 
    a= np.arange(-nnh,0, dtype = int)
    b = np.arange(1,nnh+1, dtype = int)

    NH_ID = np.concatenate((a,b))# do not include self
    #NH_ID = np.concatenate((a,0, b))% include self (i.e., "center pixel")

    if not NH_ID.any():
        # nothing to do: working with pixel only, without neighbors
        XX=X
    else:
        # stack neighbors
        XX=np.array([]) 
    for inh in NH_ID:
        XTMP= np.roll(XE,inh,axis=0) 
        if not XX.size:  
            XX = XTMP[nnh:XTMP.shape[0]-nnh-1,:]
        else:
            XX=np.hstack((XX,XTMP[nnh:XTMP.shape[0]-nnh-1,:]))
    return XX

def NLSS_torch( X, param,r_loc):
#    XSS_ij = exp( -1/param * sum_k |X(i,k) - X(j,k)| / N_features )
# size(X)= N_frames x N_features 
# size(XSS)= N_frames x N_frames 
#
# if r_loc is given, it specifies the "search radius" for NLmeans
# in this case:
#    XSS_ij is exponentially tapered
#    XSS_ij = 0 for |i-j|>r_loc
#    XSS is periodically extended to size N_frames x N_frames + 2*r_loc
    if len(sys.argv)<3:
        r_loc=[]
        
    N = X.size()[0]
    dim = X.size()[1]
    #--- compute distance & similarity
    XM2 = PDIST(X)/dim
    XSS0 = torch.exp(-1/param*XM2).double()
    # if not NLM over whole sequence, set similarity beyond search radius to
    # zero and extend boundaries
    if r_loc:   
        t1=torch.zeros(N,1).to(device)
        t1[:,0]=torch.arange(0,N)
        T=PDIST(t1)
        mparam=-1.3862943611198906/r_loc
        MASK=torch.exp(mparam*T).double()
        XSS=torch.mul(XSS0,MASK)
        XSS = XSS.transpose(1,0)
        
        np1=torch.flip(XSS[0:r_loc,:],(0,))        
        np2=torch.flip(XSS[XSS.size()[0]-r_loc:XSS.size()[0],:],(0,))
        XSS = torch.cat((np1, XSS, np2))
        XSS = XSS.transpose(1,0)
    return XSS

def PCA(X, k=2):
    # preprocess the data
    X_mean = torch.mean(X,0)
    X = X - X_mean.expand_as(X)
    # svd
    U,S,V = torch.svd(torch.t(X))
    return U[:,:k], torch.mm(X,U[:,:k])

def distx_fun(X):
    N = X.size()[0]
    dim=X.size()[1]

    Z = torch.mm(X,X.t())#.to(device)
    djj = torch.sqrt(torch.diag(Z))*torch.ones(1,N).double().t().to(device)
    Z = torch.div(1 - torch.div(torch.div(Z,djj.t()), djj) , dim)
    return Z
    
def simz_fun(Z,param):
    G = torch.exp(torch.mul(-1/param,Z)) 
    N=G.size()[1]
    z = torch.sum(G,0)
    G=torch.div(G,z.repeat(N,1));
    return G

def norm11(X):
    for i in range(0,X.size()[1]):
        X[:,i]=X[:,i]-X[:,i].min()
        mm=X[:,i].max()
        if mm is not 0:
            X[:,i]=X[:,i]/mm*2-1
    return X

def loss_fun(G,W):
    N=G.size()[0]
    # cross-entropy loss 
    w=W.contiguous().view(N*N,-1)
    g=G.contiguous().view(N*N,-1)
    L=-torch.sum(torch.mul(w,torch.log(g)))
    L=L+torch.sum(torch.mul(w,torch.log(w)))
    return L

def d_distx_fun(X):
    tt=X.size()
    N=tt[0]
    dim=tt[1]

    Z = torch.mm(X,X.t()) + 1e-16
    N = Z.size()[1];
    djj = torch.sqrt(torch.diag(Z))*torch.ones(1,N).double().t().to(device)
    dii= djj.t()
    t1 = torch.div(torch.div(1,dii), djj)
    t1 = t1 + torch.diag(torch.diag(t1)) # OK
    t2 = t1*torch.div(torch.div(Z,djj), djj) # OK

    X1=X
    X1.detach().clone()
    X1=X1.repeat((N,1,1))
    X2=X1
    X2.detach().clone()
    X2=X2.transpose(0,1)

    T1=t1.repeat((dim,1,1))
    T1=T1.transpose(0,1).transpose(1,2)
    T2=t2.repeat((dim,1,1))
    T2=T2.transpose(0,1).transpose(1,2)

    #dZ=X2*T1 - X1*T2
    dZ=torch.mul(X2,T1) - torch.mul(X1,T2)
    dZ=-dZ
    return dZ 
    
def d_simz_fun(Z,dZ,param):
    tt=dZ.size()
    N=tt[0]
    dim=tt[2]
    G = torch.exp(torch.mul(-1/param,Z))
    dG0=-1/param*G
    dG0=dG0.repeat((dim,1,1))
    dG0=dG0.transpose(0,1).transpose(1,2)
    #dG=dZ*dG0
    dG=torch.mul(dZ,dG0)

    sG=torch.sum(G,0).repeat((N,1)).repeat((dim,1,1)).transpose(0,1).transpose(1,2)
    sdG=torch.sum(dG,0).repeat((N,1,1))

    repG=G.repeat((dim,1,1)).transpose(0,1).transpose(1,2)
    dG=torch.div((dG*sG - repG*sdG) , (sG*sG))
    return dG

def d_loss_fun(G,dG,W):
    tt=dG.size()
    N=tt[0]
    dim=tt[2]
    WG=torch.div(W,G).repeat((dim,1,1)).transpose(0,1).transpose(1,2)
    dL=-torch.sum(dG*WG,0)
    return dL

def full_grad_fun(X,W,param):
    Z = distx_fun(X)
    dZ = d_distx_fun(X)
    G = simz_fun(Z,param)
    dG = d_simz_fun(Z,dZ,param)
    dL = d_loss_fun(G,dG,W)
    return dL, G

def sim_grad_descent(X,W,param,niter):
    eta0=1e-6
    N=X.size()[0]
    dim=X.size()[1]
    # step 1
    dL,G=full_grad_fun(X,W,param) # compute gradient
    x1=X.permute(1,0)
    g1=dL.permute(1,0)
    x1=x1.contiguous().view(-1,N*dim)
    g1=g1.contiguous().view(-1,N*dim)
    eta=eta0
    dataiter=X-eta*dL;

    # step 2-niter
    for it in range(niter-1):
        dL,G=full_grad_fun(dataiter,W,param) # compute gradient
        x2=x1
        g2=g1
        x2.detach().clone()
        g2.detach().clone()
        x1=dataiter.permute(1,0)
        g1=dL.permute(1,0)
        x1=x1.contiguous().view(-1,N*dim)
        g1=g1.contiguous().view(-1,N*dim)
        dx=x1-x2
        dg=g1-g2;
        eta=torch.mm(dg,dx.transpose(1,0)) / torch.mm(dg,dg.transpose(1,0))

        if eta != eta: # eta is nan. this actually means that the gradient is zero: STOP
            #print(g1)
            eta=1e-16
        
        dataiter=dataiter - eta*dL
    
    return dataiter

def sim_grad_descent_autograd(X,W,param,niter):
    eta0=1e-6
    N=X.size()[0]
    dim=X.size()[1]
    # step 1
    
    loss=loss_fun(simz_fun(distx_fun(X),param),W)
    loss.backward()
    dL=X.grad

    x1=X.permute(1,0)
    g1=dL.permute(1,0)
    x1=x1.contiguous().view(-1,N*dim)
    g1=g1.contiguous().view(-1,N*dim)
    eta=eta0
    X=X-eta*dL
    X=X.detach().clone()
    X.requires_grad=True

    # step 2-niter
    for it in range(niter-1):
        loss=loss_fun(simz_fun(distx_fun(X),param),W)
        loss.backward()
#        dL=X.grad.detach().clone()
        dL=X.grad
        x2=x1
        g2=g1
        x2.detach().clone()
        g2.detach().clone()
        x1=X.permute(1,0)
        g1=dL.permute(1,0)
        x1=x1.contiguous().view(-1,N*dim)
        g1=g1.contiguous().view(-1,N*dim)
        dx=x1-x2
        dg=g1-g2
        eta=torch.mm(dg,dx.transpose(1,0)) / torch.mm(dg,dg.transpose(1,0))

        if eta != eta: # eta is nan. this actually means that the gradient is zero: STOP
            #print(g1)
            eta=1e-16
        
        X=X - eta*dL
        X=X.detach().clone()
        X.requires_grad=True
    
    return X

def sim_grad_descent_biW(X,W1,W2,param,aa,niter):
    eta0=1e-6
    aa=min(max(aa,0),1)
    a1,a2=aa,1-aa
    N=X.size()[0]
    dim=X.size()[1]
    # step 1
    dL1,G1=full_grad_fun(X,W1,param) # compute gradient
    dL2,G2=full_grad_fun(X,W2,param) # compute gradient
    dL=a1*dL1+a2*dL2
    G=a1*G1+a2*G2
    x1=X.permute(1,0)
    g1=dL.permute(1,0)
    x1=x1.contiguous().view(-1,N*dim)
    g1=g1.contiguous().view(-1,N*dim)
    eta=eta0
    dataiter=X-eta*dL;

    # step 2-niter
    for it in range(niter-1):
        dL1,G1=full_grad_fun(dataiter,W1,param) # compute gradient
        dL2,G2=full_grad_fun(dataiter,W2,param) # compute gradient
        dL=a1*dL1+a2*dL2
        G=a1*G1+a2*G2
        x2=x1
        g2=g1
        x2.detach().clone()
        g2.detach().clone()
        x1=dataiter.permute(1,0)
        g1=dL.permute(1,0)
        x1=x1.contiguous().view(-1,N*dim)
        g1=g1.contiguous().view(-1,N*dim)
        dx=x1-x2
        dg=g1-g2;
        eta=torch.mm(dg,dx.transpose(1,0)) / torch.mm(dg,dg.transpose(1,0))

        if eta != eta: # eta is nan. this actually means that the gradient is zero: STOP
            #print(g1)
            eta=1e-16
        
        dataiter=dataiter - eta*dL
    
    return dataiter

def loss_fun_biW(G,W1,W2,aa):
    N=G.size()[0]
    aa=min(max(aa,0),1)
    a1,a2=aa,1-aa
    # cross-entropy loss 
    w1=W1.contiguous().view(N*N,-1)
    w2=W2.contiguous().view(N*N,-1)
    g=G.contiguous().view(N*N,-1)
    L=-a1*torch.sum(torch.mul(w1,torch.log(g)))-a2*torch.sum(torch.mul(w2,torch.log(g)))
    L=L+a1*torch.sum(torch.mul(w1,torch.log(w1)))+a2*torch.sum(torch.mul(w2,torch.log(w2)))
    return L

def sim_grad_descent_biW_autograd(X,W1,W2,param,aa,niter):
    eta0=1e-6
    N=X.size()[0]
    dim=X.size()[1]
    # step 1
    
    loss=loss_fun_biW(simz_fun(distx_fun(X),param),W1,W2,aa)
    loss.backward()
    dL=X.grad

    x1=X.permute(1,0)
    g1=dL.permute(1,0)
    x1=x1.contiguous().view(-1,N*dim)
    g1=g1.contiguous().view(-1,N*dim)
    eta=eta0
    X=X-eta*dL
    X=X.detach().clone()
    X.requires_grad=True

    # step 2-niter
    for it in range(niter-1):
        loss=loss_fun_biW(simz_fun(distx_fun(X),param),W1,W2,aa)
        loss.backward()
        dL=X.grad
        x2=x1
        g2=g1
        x2.detach().clone()
        g2.detach().clone()
        x1=X.permute(1,0)
        g1=dL.permute(1,0)
        x1=x1.contiguous().view(-1,N*dim)
        g1=g1.contiguous().view(-1,N*dim)
        dx=x1-x2
        dg=g1-g2
        eta=torch.mm(dg,dx.transpose(1,0)) / torch.mm(dg,dg.transpose(1,0))

        if eta != eta: # eta is nan. this actually means that the gradient is zero: STOP
            #print(g1)
            eta=1e-16
        
        X=X - eta*dL
        X=X.detach().clone()
        X.requires_grad=True
    
    return X

In [4]:
FPTH = "/media/DATA/DATASET/EDUB-Seg/Feat_2048/"
SPTH = FPTH + 'NL_SS_FINAL/self_loc_L1_'

SAVEDIR = ""
SAVEDIR = ""

N_SEQ=20
SNME='EDUB-Seg'


DO_AUTOGRAD=0 # Use Autograd instead of manual grad
DO_PCA=0 # initialize embedding with PCA


# master parameter for similarity
par00=0.25 

# search radius and neighborhood for NLmean
r_loc=3
nnh=1

# parameters for initial embedding
param2=par00/0.8
param1=par00/50/2
d_emb = 15
niter_e=150

# parameters for dynamic graph embedding
N_iter_G=10 # number of DGE iterations
niter_g=5  # embedding steps
aa=0.3     # proportionality factor for initial matrix W0
aa2=1e-3   # strengthen diagonal
NCL=9      # number of clusters
Cdamp=0.95 # reduction of out-of-cluster similarity




aa=0.3; aa2=3e-2; N_iter_G=10; NCL=10; niter_g=5; Cdamp=0.975;
aa2=3e-1; 

aa=0.1; aa2=5e-1; N_iter_G=10; NCL=10; niter_g=5; Cdamp=0.85;



aa=0.4; aa2=3e-1; N_iter_G=10; NCL=10; niter_g=5; Cdamp=0.9; NF=1;
#_a0.4_t0.3_NF1_CL10_CLd0.9_ie5

aa=0.3; aa2=3e-1; N_iter_G=10; NCL=10; niter_g=3; Cdamp=0.9; NF=2;
#_a0.3_t0.3_NF2_CL10_CLd0.9_ie3

aa=0.1; aa2=3e-1; N_iter_G=10; NCL=10; niter_g=3; Cdamp=0.9; NF=3;
#_a0.1_t0.3_NF3_CL10_CLd0.9_ie3




SAVE_OUT=1

#KMEANS_sklearn=0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# FORCE CPU for timing purpose
#device = torch.device("cpu")

NLSstr= '_NH' + str(nnh) + '_R' + str(r_loc) + '_'+str(par00)
embstr = '_emb_dim' + str(d_emb)+'_it' + str(niter_e)

print(device)

cuda


In [1]:
from scipy import signal



for subid in range(20): #range(N_SEQ): 
    

    
    #--- LOAD EDUB features
    FILENAME= FPTH + SNME
    FN= SNME + '_seq' + str(subid+1) 
    X1 = np.load(FILENAME + '_seq' + str(subid+1) + '.npy')

    N = X1.shape[0]; print(FN, ' N =', N)
    
    #######################################################
    #--- PART 1: NLSS : compute similarity and do nonlocal mean
    
    if DO_PCA==1:
        PCAstr = '_PCA'
    else:
        PCAstr = ''
        
    if DO_AUTOGRAD==1:
        GST='_auto'
    else:
        GST='_man'

    try:
        FN1 = SAVEDIR + FN + NLSstr + '.npy'; 
        dataiter = np.load(FN1)
    except:
        # - compute (self)-similarity 
        X1=X1-X1.min(); X1=X1/X1.max()*2-1 #normalized to [-1,1]
        XXT=torch.tensor(construct_SS_matrix(X1,nnh)).to(device) # stack neighborhood
        XSS = NLSS_torch(XXT,par00,r_loc).cpu().detach().numpy() # self-similarity
        X1r=np.concatenate((np.flipud(X1[0:r_loc,:]),X1, np.flipud(X1[X1.shape[0]-r_loc:X1.shape[0],:]))) # pad with search radius (periodic boundary conditions)

        # - compute non-local mean
        a = np.matmul(np.asarray(XSS),np.asarray(X1r))
        xss = np.transpose(XSS)
        b = np.diag(1./xss.sum(axis=0))
        dataiter=np.matmul(b,a) # non-local mean
        dataiter=dataiter-dataiter.min(); dataiter=dataiter/dataiter.max()*2-1 #----- normalize to [-1,1];

        if SAVE_OUT==1:
            FN1 = SAVEDIR + FN + NLSstr + '.npy'; 
            np.save(FN1,dataiter)
        #        disave=norm11(dataiter); np.save(FN1,disave)
    
    # - initial similarity of cleaned data
    dataiter = torch.from_numpy(dataiter).to(device)
    dataiter = norm11(dataiter)
    W1 = simz_fun(distx_fun(dataiter),param1)
        
    #######################################################
    #--- PART 2: EMBEDDING
    try:
#        # PROBLEM WITH PCA
#        aaa
        FN2 = SAVEDIR + FN + NLSstr + embstr + GST + PCAstr + '.npy'; 
        dataiter = np.load(FN2)
        dataiter = torch.from_numpy(dataiter).to(device)
    except:
        # - send cleaned data to device and normalize each feature
        #dataiter = torch.from_numpy(dataiter).to(device)
        dataiter = norm11(dataiter)

        # - initial similarity of cleaned data
        W1 = simz_fun(distx_fun(dataiter),param1) # OK

        # - initialize
        if DO_PCA==1:
            LF, dataiter = PCA(dataiter,d_emb)
        dataiter = norm11(dataiter[:,:d_emb])

        # - 1st order LINE
        #dataiter.cpu().detach().numpy()
        if DO_AUTOGRAD==1:
            dataiter.requires_grad=True # required for AUTOGRAD
            dataiter = sim_grad_descent_autograd(dataiter,W1,param2,niter_e)
            dataiter=dataiter.detach()
            GST='_auto'
        else:
            dataiter = sim_grad_descent(dataiter,W1,param2,niter_e)
            GST='_man'

        if SAVE_OUT==1:
            FN2 = SAVEDIR + FN + NLSstr + embstr + GST + PCAstr + '.npy'; np.save(FN2,(dataiter).cpu().detach().numpy())
        
    FN1 = SAVEDIR + FN + NLSstr + '.npy'
    FN2 = SAVEDIR + FN + NLSstr + embstr + GST + PCAstr + '.npy'
    print(FN1)
    print(FN2)
        
    
    #######################################################
    #--- PART 3: DGE LOOP
    
    # - normalize and initialize
    dataiter = norm11(dataiter).double()
    #--- initial data similarity
    W2 = simz_fun(distx_fun(dataiter),param2)
    
    #######################################################
    y = np.linspace(1, N, N)
    xv, yv = np.meshgrid(y, y)
    id1=np.where(abs(xv-yv)>1)
    id2=np.where(abs(xv-yv)<2)
    #######################################################
    
    #######################################################
    N=dataiter.shape[0]
    y = np.linspace(1, N, N)
    xv, yv = np.meshgrid(y, y)
    id1=np.where(abs(xv-yv)>1)
    id2=np.where(abs(xv-yv)<2)
    iNF=int(NF)
    hNF=int(NF/2)
    FF= Variable(torch.ones(1, 1, iNF, iNF)/(NF*NF)).double().to(device)
#                F=np.ones((int(NF),int(NF)))/(NF*NF)
    #######################################################
    
    for nDGE in range(N_iter_G):
        
        #########################################################################
        ####### !!!!! CHANGED TEMPORAL REGULARIZATION !!!!! HW 22/07/2019 #######
        #########################################################################

        #--- 0 modify W: temporal smoothing
        if int(NF)>1:
            W = torch.zeros(1, 1, N+iNF, N+iNF).double().to(device)
            W[0,0,hNF:N+hNF,hNF:N+hNF]=W2
            W2 = FTR.conv2d(W, FF)[0,0,:N,:N].double()
#            F=np.ones((NF,NF))/(NF*NF)
#            W2= torch.from_numpy(signal.convolve2d(W2.cpu().detach().numpy(), F, boundary='symm', mode='same')).to(device)
        
        #--- 0 modify W: strengthen diagonal
        #W2=W2*(torch.eye(W2.size()[0]).double().to(device)*aa2 + torch.ones(W2.size()[0]).double().to(device)*(1-aa2))
        W2[id1[0],id1[1]]=(1-aa2)*W2[id1[0],id1[1]]
        

        #%W=diag(diag(W))+(1-aa2)*(W-diag(diag(W)));
        #[i1,i2]=meshgrid(1:N); W(abs(i1-i2)>1)= (1-aa2)*W(abs(i1-i2)>1);
        
        #--- 1 update embedding + sim matrix
        if DO_AUTOGRAD==1:
            dataiter.requires_grad=True
            dataiter = sim_grad_descent_biW_autograd(dataiter,W1,W2,param2,aa,niter_g).detach().clone()
        else:
            dataiter = sim_grad_descent_biW(dataiter,W1,W2,param2,aa,niter_g)
        dataiter = norm11(dataiter)
        
        #--- 2 compute new W
        Z = distx_fun(dataiter) 
        W2 = simz_fun(Z,param2)
        
        #--- 3 modify W from clustering
        #kmeans = KMeans(n_clusters=NCL, random_state=0).fit(dataiter.cpu().numpy())
        kmeans = KMeans(n_clusters=NCL, random_state=0, init='k-means++').fit(dataiter.cpu().numpy())
        #kmeans = KMeans(n_clusters=NCL).fit(dataiter.cpu().numpy()) ## RANDOM ##
        CC0=torch.tensor([kmeans.labels_]).transpose(1,0)

        CIM = class_sim_torch(CC0).to(device)
        CIM = (CIM==0).double() + Cdamp*(CIM!=0).double()
        W2=W2*CIM
        
        if SAVE_OUT==1:
            FN3 = SAVEDIR + FN + NLSstr + embstr + GST + PCAstr + '_it' + str(nDGE+1) + '_DGE.npy'; 
            if iNF>1:
                FN3 = SAVEDIR + FN + NLSstr + embstr + GST +'_NF' + str(iNF) + PCAstr + '_it' + str(nDGE+1) + '_DGE.npy'; 
            np.save(FN3,dataiter.cpu().detach().numpy())


    
    #######################################################

    # --- save
    
#    np.save(SPTH+SaveStr,dataiter)



NameError: name 'FPTH' is not defined

In [ ]:
NF=3
F=np.ones((NF,NF))/(NF*NF)
print(F)

In [ ]:
import torch
import torch.nn.functional as F
from torch.autograd import Variable
N = 5
NF=3.0

iNF=int(NF)
hNF=int(NF/2)
W0 = torch.ones( N, N)
W = torch.zeros(1, 1, N+iNF, N+iNF)
W[0,0,hNF:N+hNF,hNF:N+hNF]=W2

FF= Variable(torch.ones(1, 1, iNF, iNF) )#/(NF*NF))
W2 = F.conv2d(W, FF)[0,0,:N,:N]

In [ ]:
torch.cuda.empty_cache()